In [2]:
import json
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    DataCollatorForSeq2Seq, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer
)
from pathlib import Path
import os
import gdown
import dotenv
dotenv.load_dotenv()

ImportError: cannot import name 'in1d' from 'numpy.lib._arraysetops_impl' (d:\Afaculta\Master\NMT\Ro-Md-En-MT-Analysis\.venv\Lib\site-packages\numpy\lib\_arraysetops_impl.py)

In [51]:
# only run on colab
%pip install sacremoses

In [53]:
!git clone https://github.com/TanaseVictorFlavian/Ro-Md-En-MT-Analysis.git

Cloning into 'Ro-Md-En-MT-Analysis'...
remote: Enumerating objects: 17839, done.
remote: Counting objects: 100% (17839/17839), done.
remote: Compressing objects: 100% (17822/17822), done.
remote: Total 17839 (delta 8), reused 17835 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (17839/17839), 4.46 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
# Loading Data
parallel_corpus_dir = Path.cwd() / "parallel_corpus"

if parallel_corpus_dir.exists() is False:
    # fallback for using colab extension in vscode
    parallel_corpus_dir = Path("/content/Ro-Md-En-MT-Analysis/parallel_corpus")

json_files = list(parallel_corpus_dir.glob("*.json"))
data_list = []

for file_path in json_files:
    try:
        content = file_path.read_text(encoding='utf-8')
        data_list.append(json.loads(content))
        # am scos chestia cu index = 1000 
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

print(len(json_files)) # sanity check

In [ ]:
raw_dataset = Dataset.from_list(data_list).shuffle(seed=2002) # am adaugat shuffle ca sa nu mai avem prima jumatate ro a doua md
split_datasets = raw_dataset.train_test_split(test_size=0.2, seed=42) 

dataset = DatasetDict({
    'train': split_datasets['train'],
    'dev': split_datasets['test']
})

In [ ]:
raw_subset = raw_dataset.select(range(0, 1000))
split_subset = raw_subset.train_test_split(test_size=0.2, seed=42)

subset = DatasetDict({
    "train":split_subset['train'],
    "test":split_subset['test']
})

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [61]:
model_checkpoint = "Helsinki-NLP/opus-mt-ROMANCE-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples["source"]]
    targets = [ex for ex in examples["target"]]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_subsets = subset.map(preprocess_function, batche=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [63]:
args = Seq2SeqTrainingArguments(
    output_dir="./opus-mt-finetuned",
    eval_strategy="epoch",      
    save_strategy="epoch", 
    load_best_model_at_end=True,      
    metric_for_best_model="eval_loss",
    greater_is_better=False,         
    save_total_limit=1,            
    learning_rate=2e-5,               
    per_device_train_batch_size=32,   
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    num_train_epochs=10,                
    fp16=torch.cuda.is_available(),    
    report_to="none",            
    logging_strategy="epoch",     
    logging_steps=1,                 
    logging_dir=None,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Starting training...")
trainer.train()
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")
print("Training complete. Model saved to './final_model'")

/tmp/ipython-input-1729137930.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Starting training...


Epoch,Training Loss,Validation Loss
1,0.860200,0.790198
2,0.651000,0.768445
3,0.541200,0.757784
4,0.461400,0.759584
5,0.409500,0.760808
6,0.363500,0.764995
7,0.327200,0.766895
8,0.307500,0.769391
9,0.294100,0.770834
10,0.284900,0.771506


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Training complete. Model saved to './final_model'


In [80]:
!zip -r model_output.zip /content/final_model


  adding: content/final_model/ (stored 0%)
  adding: content/final_model/special_tokens_map.json (deflated 35%)
  adding: content/final_model/tokenizer_config.json (deflated 62%)
  adding: content/final_model/training_args.bin (deflated 53%)
  adding: content/final_model/config.json (deflated 63%)
  adding: content/final_model/source.spm (deflated 49%)
  adding: content/final_model/target.spm (deflated 50%)
  adding: content/final_model/vocab.json (deflated 70%)
  adding: content/final_model/model.safetensors (deflated 7%)
  adding: content/final_model/generation_config.json (deflated 40%)


In [ ]:
input_text = "Merg la piață să iau pepene." 
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Source: {input_text}")
print(f"Translation: {decoded}")